`Note`: Before running or testing the code in this notebook, ensure that you have set up the `Zep server`. 

# 🧠 Zep Memory Integration with LlamaIndex Agents

This notebook demonstrates how to use [Zep memory](https://docs.getzep.com/) with various agent types from LlamaIndex, including:

- `SimpleChatEngine`
- `ReActAgent`
- `FunctionCallingAgent`
- `AgentWorkflow`

Both **sync** and **async** memory clients are covered.


## Install Dependencies

In [ ]:
# %pip install llama-index zep-python openai

## Environment Setup

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxxx"

## Import Required Packages

In [ ]:
import uuid
from zep_python.client import Zep, AsyncZep
from llamaindex.memory.zep import ZepMemory
from llama_index.llms.openai import OpenAI

## Initialize Clients and IDs

In [ ]:
zep_client = Zep(api_key="mysupersecretkey", base_url="http://localhost:8000")
azep_client = AsyncZep(api_key="mysupersecretkey", base_url="http://localhost:8000")

In [ ]:
user_id = uuid.uuid4().hex  # A new user identifier
new_user = zep_client.user.add(
    user_id=user_id,
)

# create a chat session
session_id = uuid.uuid4().hex  # A new session identifier
session = zep_client.memory.add_session(
    session_id=session_id,
    user_id=user_id,
)

## Initialize Memory (Sync and Async)

In [ ]:
memory = ZepMemory.from_defaults(
    zep_client=zep_client,  # Zep client
    session_id=session_id,  # Optional: provide a session ID or one will be generated
    user_id=user_id,  # Optional: provide a user ID for user-specific context
)

amemory = ZepMemory.from_defaults(
    zep_client=azep_client,  # AsyncZep client
    session_id=session_id,  # Optional: provide a session ID or one will be generated
    user_id=user_id,  # Optional: provide a user ID for user-specific context
)

/home/younis/Desktop/dev/llama_index/llama-index-integrations/memory/llama-index-memory-zep/llamaindex/memory/zep/base.py:39: RuntimeWarning: coroutine 'AsyncMemoryClient.get' was never awaited
  self._sync_from_zep()


## LLM Setup

In [ ]:
llm = OpenAI(model="gpt-4o-mini")  # You can swap this with other supported LLMs

## SimpleChatEngine Demo

In [ ]:
from llama_index.core.chat_engine.simple import SimpleChatEngine

In [ ]:
agent = SimpleChatEngine.from_defaults(llm=llm, memory=memory)  # set you memory here

# Start the chat
response = agent.chat("Hi, My name is Younis")
print(response)

# Now test memory retention:
response = agent.chat("What was my name?")
print(response)

Hello again, Younis! How can I help you today?
Your name is Younis.


In [ ]:
agent = SimpleChatEngine.from_defaults(llm=llm, memory=amemory)  # set you memory here

# Start the chat
response = agent.chat("Hi, My name is Younis")
print(response)

# Now test memory retention:
response = agent.chat("What was my name?")
print(response)

Hello, Younis! How can I assist you today?
Your name is Younis.


## React Agent  Demo 

In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    tools=[],
    llm=llm,
    memory=memory,
    verbose=True,
)

In [ ]:
response = agent.chat("What's the capital of France?")
print(response)

# Now test memory retention:
response = agent.chat("What was my previous question?")
print(response)

> Running step 9be3ba30-3d10-4617-844b-124ae0e5f17b. Step input: What's the capital of France?
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: La capitale de la France est Paris.
```
La capitale de la France est Paris.
```
> Running step 0efcf470-f5b1-4aca-94c2-91497aa3ce39. Step input: What was my previous question?
Thought: The current language of the user is: English. I need to answer the question without using any tools.
Answer: Your previous question was "What's the capital of France?"
Your previous question was "What's the capital of France?"


In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    tools=[],
    llm=llm,
    memory=amemory,
    verbose=True,
)

In [ ]:
response = agent.chat("What's the capital of France?")
print(response)

# Now test memory retention:
response = agent.chat("What was my previous question?")
print(response)

> Running step a31ec9cc-0d68-4948-b413-1bc0fee98611. Step input: What's the capital of France?


Thought: The current language of the user is: English. I can answer without using any more tools.
Answer: The capital of France is Paris.
The capital of France is Paris.
> Running step 8be75b08-36bd-4520-b23d-b62b06fec08f. Step input: What was my previous question?
Thought: (Implicit) I can answer without any more tools!
Answer: Your previous question was, "What's the capital of France?"
Your previous question was, "What's the capital of France?"


## FunctionCallingAgent Demo

In [ ]:
from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(
    [],
    llm=llm,
    memory=memory,
    verbose=True,
)

In [ ]:
# Start the chat
response = agent.chat("Hi, My name is Younis")
print(response)

# Now test memory retention:
response = agent.chat("What was my name?")
print(response)

> Running step 08a916cc-f2d1-4f5a-9b7a-87cba46b93dd. Step input: Hi, My name is Younis
Added user message to memory: Hi, My name is Younis
=== LLM Response ===
Hello, Younis! How can I assist you today?
Hello, Younis! How can I assist you today?
> Running step ee67d336-b260-4ea4-885d-f280b731ea6b. Step input: What was my name?
Added user message to memory: What was my name?
=== LLM Response ===
Your name is Younis.
Your name is Younis.


In [ ]:
from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(
    [],
    llm=llm,
    memory=amemory,
    verbose=True,
)

In [ ]:
# Start the chat
response = agent.chat("Hi, My name is Younis")
print(response)

# Now test memory retention:
response = agent.chat("What was my name?")
print(response)

> Running step 6a92d2f6-a596-49d6-a1e8-62533eae3baa. Step input: Hi, My name is Younis
Added user message to memory: Hi, My name is Younis
=== LLM Response ===
Hello again, Younis! How can I help you today?
Hello again, Younis! How can I help you today?
> Running step 712ec8b5-c820-4364-92d1-0ccaa81da621. Step input: What was my name?
Added user message to memory: What was my name?
=== LLM Response ===
Your name is Younis.
Your name is Younis.


##  AgentWorkflow Demo

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

In [ ]:
# Create a ResearchAgent with a system prompt that guides it to be systematic
research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Responsible for generating well-structured responses based on internal knowledge and context.",
    system_prompt="""
    You are the ResearchAgent. Your task is to compile and synthesize information based solely on the provided context.
    Work in a systematic, transparent manner, explaining your thought process and summarizing the key insights clearly.
    """,
    llm=llm,
    tools=[],
    verbose=True,
)

# Define the agent workflow with the ResearchAgent as the root agent
agent_workflow = AgentWorkflow(
    agents=[research_agent],
    root_agent=research_agent.name,
    initial_state={"answer_content": ""},
)

In [ ]:
# await ensure_weaviate_connection(aclient)

handler = agent_workflow.run(
    user_msg="Explain the heuristic function in detail.", memory=memory
)

current_agent = None
tool_output_buffer = ""

async for event in handler.stream_events():
    # Handle agent changes with clear visual separation
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # Stream all content from AgentStream events in real-time
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)


🤖 Agent: ResearchAgent

To explain the heuristic function in detail, I will break down the concept into several key components: definition, purpose, types, and examples.

### Definition
A heuristic function is a method used in algorithms, particularly in search and optimization problems, to estimate the cost or distance from a given state to a goal state. It provides a way to guide the search process by evaluating which paths are more promising based on certain criteria.

### Purpose
The primary purpose of a heuristic function is to improve the efficiency of search algorithms. By providing an estimate of the cost to reach the goal, the heuristic helps the algorithm prioritize which nodes to explore first. This can significantly reduce the number of nodes that need to be evaluated, leading to faster solutions.

### Types of Heuristic Functions
1. **Admissible Heuristic**: A heuristic is admissible if it never overestimates the cost to reach the goal. This property ensures that the algo

In [ ]:
# await ensure_weaviate_connection(aclient)

handler = agent_workflow.run(
    user_msg="Explain the heuristic function in detail.", memory=amemory
)

current_agent = None
tool_output_buffer = ""

async for event in handler.stream_events():
    # Handle agent changes with clear visual separation
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # Stream all content from AgentStream events in real-time
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)


🤖 Agent: ResearchAgent

To explain the heuristic function in detail, I will provide a comprehensive overview, including its definition, purpose, characteristics, types, and applications, particularly in the context of search algorithms.

### Definition
A heuristic function, often denoted as \( h(n) \), is a function that estimates the cost or distance from a given node \( n \) to the goal node in a search space. It is used in various algorithms to guide the search process toward the most promising paths.

### Purpose
The main purpose of a heuristic function is to improve the efficiency of search algorithms by providing an estimate of how far a node is from the goal. This allows the algorithm to prioritize which nodes to explore first, thereby reducing the overall search time and space.

### Characteristics of Heuristic Functions
1. **Admissibility**: A heuristic is admissible if it never overestimates the actual cost to reach the goal from any node. This property ensures that the sear